<a href="https://colab.research.google.com/github/CaesarXInsanium/TENSORFLOW2_PERSONALPROJECTS/blob/master/tensorflow2_dqn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import collections
import cv2
import numpy as np
import matplotlib.pyplot as plt
import gym


def plot_learning_curve(x, scores, epsilons, filename, lines=None):
    fig=plt.figure()
    ax=fig.add_subplot(111, label="1")
    ax2=fig.add_subplot(111, label="2", frame_on=False)

    ax.plot(x, epsilons, color="C0")
    ax.set_xlabel("Training Steps", color="C0")
    ax.set_ylabel("Epsilon", color="C0")
    ax.tick_params(axis='x', colors="C0")
    ax.tick_params(axis='y', colors="C0")

    N = len(scores)
    running_avg = np.empty(N)
    for t in range(N):
	    running_avg[t] = np.mean(scores[max(0, t-20):(t+1)])

    ax2.scatter(x, running_avg, color="C1")
    ax2.axes.get_xaxis().set_visible(False)
    ax2.yaxis.tick_right()
    ax2.set_ylabel('Score', color="C1")
    ax2.yaxis.set_label_position('right')
    ax2.tick_params(axis='y', colors="C1")

    if lines is not None:
        for line in lines:
            plt.axvline(x=line)

    plt.savefig(filename)

class RepeatActionAndMaxFrame(gym.Wrapper):
    def __init__(self, env=None, repeat=4, clip_reward=False, no_ops=0,
                 fire_first=False):
        super(RepeatActionAndMaxFrame, self).__init__(env)
        self.repeat = repeat
        self.shape = env.observation_space.low.shape
        self.frame_buffer = np.zeros_like((2, self.shape))
        self.clip_reward = clip_reward
        self.no_ops = no_ops
        self.fire_first = fire_first

    def step(self, action):
        t_reward = 0.0
        done = False
        for i in range(self.repeat):
            obs, reward, done, info = self.env.step(action)
            if self.clip_reward:
                reward = np.clip(np.array([reward]), -1, 1)[0]
            t_reward += reward
            idx = i % 2
            self.frame_buffer[idx] = obs
            if done:
                break

        max_frame = np.maximum(self.frame_buffer[0], self.frame_buffer[1])
        return max_frame, t_reward, done, info

    def reset(self):
        obs = self.env.reset()
        no_ops = np.random.randint(self.no_ops)+1 if self.no_ops > 0 else 0
        for _ in range(no_ops):
            _, _, done, _ = self.env.step(0)
            if done:
                self.env.reset()
        if self.fire_first:
            assert self.env.unwrapped.get_action_meanings()[1] == 'FIRE'
            obs, _, _, _ = self.env.step(1)

        self.frame_buffer = np.zeros_like((2,self.shape))
        self.frame_buffer[0] = obs

        return obs

class PreprocessFrame(gym.ObservationWrapper):
    def __init__(self, shape, env=None):
        super(PreprocessFrame, self).__init__(env)
        self.shape = (shape[2], shape[0], shape[1])
        self.observation_space = gym.spaces.Box(low=0.0, high=1.0,
                                    shape=self.shape, dtype=np.float32)

    def observation(self, obs):
        new_frame = cv2.cvtColor(obs, cv2.COLOR_RGB2GRAY)
        resized_screen = cv2.resize(new_frame, self.shape[1:],
                                    interpolation=cv2.INTER_AREA)
        new_obs = np.array(resized_screen, dtype=np.uint8).reshape(self.shape)
        new_obs = new_obs / 255.0

        return new_obs

class StackFrames(gym.ObservationWrapper):
    def __init__(self, env, repeat):
        super(StackFrames, self).__init__(env)
        self.observation_space = gym.spaces.Box(
                            env.observation_space.low.repeat(repeat, axis=0),
                            env.observation_space.high.repeat(repeat, axis=0),
                            dtype=np.float32)
        self.stack = collections.deque(maxlen=repeat)

    def reset(self):
        self.stack.clear()
        observation = self.env.reset()
        for _ in range(self.stack.maxlen):
            self.stack.append(observation)

        return np.array(self.stack).reshape(self.observation_space.low.shape)

    def observation(self, observation):
        self.stack.append(observation)

        return np.array(self.stack).reshape(self.observation_space.low.shape)

def make_env(env_name, shape=(84,84,1), repeat=4, clip_rewards=False,
             no_ops=0, fire_first=False):
    env = gym.make(env_name)
    env = RepeatActionAndMaxFrame(env, repeat, clip_rewards, no_ops, fire_first)
    env = PreprocessFrame(shape, env)
    env = StackFrames(env, repeat)

    return env


In [0]:
import numpy as np

class ReplayBuffer(object):
    def __init__(self, max_size, input_shape, n_actions):
        self.mem_size = max_size
        self.mem_cntr = 0
        self.state_memory = np.zeros((self.mem_size, *input_shape),
                                     dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size, *input_shape),
                                         dtype=np.float32)

        self.action_memory = np.zeros(self.mem_size, dtype=np.int64)
        self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
        self.terminal_memory = np.zeros(self.mem_size, dtype=np.uint8)

    def store_transition(self, state, action, reward, state_, done):
        index = self.mem_cntr % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.action_memory[index] = action
        self.reward_memory[index] = reward
        self.terminal_memory[index] = done
        self.mem_cntr += 1

    def sample_buffer(self, batch_size):
        max_mem = min(self.mem_cntr, self.mem_size)
        batch = np.random.choice(max_mem, batch_size, replace=False)

        states = self.state_memory[batch]
        actions = self.action_memory[batch]
        rewards = self.reward_memory[batch]
        states_ = self.new_state_memory[batch]
        terminal = self.terminal_memory[batch]

        return states, actions, rewards, states_, terminal


In [0]:
class ReplayMem(object):
    def __init__(self, max_size, input_shape, n_actions):
        self.mem_size = max_size
        self.mem_cntr = 0
        self.state_mem = tf.zeros((self.mem_size, *input_shape), dtype=tf.float32)
        self.new_state_mem = tf.zeros((self.mem_size, *input_shape), dtype=tf.float32)
        self.action_mem = tf.zeros((self.mem_size), dtype=tf.int32)
        self.reward_mem = tf.zeros((self.mem_size), dtype=tf.int32)
        self.term_mem = tf.zeros((self.mem_size), dtype=tf.int16)

    def store_transition(self, state, action, reward, state_, done: bool):
        index = self.mem_cntr % self.mem_size
        self.state_mem = tf.concat([self.state_mem[1:], [state]], axis=0)
        self.new_state_mem = tf.concat([self.new_state_mem[1:], [state_]], axis=0)
        self.action_mem = tf.concat([self.action_mem[1:], [action]], axis=0)
        self.reward_mem = tf.concat([self.reward_mem[1:], [int(reward)]], axis=0)
        self.reward_mem = tf.concat([self.term_mem[1:], [done]], axis=0)
        self.mem_cntr += 1

    def sample_buffer(self, batch_size):
        batch = [[i] for i in range(batch_size)]
        states = tf.gather_nd(tf.random.shuffle(self.state_mem), batch)
        actions = tf.gather_nd(tf.random.shuffle(self.action_mem), batch)
        rewards = tf.gather_nd(tf.random.shuffle(self.reward_mem), batch)
        states_ = tf.gather_nd(tf.random.shuffle(self.new_state_mem), batch)
        terminal = tf.gather_nd(tf.random.shuffle(self.term_mem), batch)

        return states, actions, rewards, states_, terminal



In [0]:
import tensorflow as tf
from tensorflow import keras
import os

#need a gpu

class DQN(keras.Model):
    def __init__(self,lr,n_actions,name,input_dims,chkpt_dir):
        super(DQN, self).__init__()
        self.checkpoint_dir = chkpt_dir
        self.checkpoint_file = os.path.join(self.checkpoint_dir, name)
        #by default conv2D uses format (NWHC): (batch, width,height,channels)
        #format ncwh requires a gpu at the moment that is why we are here at google colab
        
        self.conv1 = keras.layers.Conv2D(input_shape=(-1,*input_dims),
            filters=32,kernel_size=8,data_format='channels_first',strides=4,activation='relu')
        #need the channels first data format in order to use the shape(32,4,84,84) format NCWH
        # but now the problem becomes that i actually need a GPU, which is bad for now but I can live with it
        #testing in google colab is positive
        self.conv2 = keras.layers.Conv2D(filters=32,kernel_size=4,strides=2,data_format='channels_first',
            activation='relu')
        self.conv3 = keras.layers.Conv2D(filters=64,kernel_size=3,strides=1,data_format='channels_first',
            activation='relu')

        self.flat = keras.layers.Flatten()
        self.dense1 = keras.layers.Dense(units=512, activation='relu')
        self.dense2 = keras.layers.Dense(units=n_actions)

        #make prepareations in order to move operatiions to proper device. for local runtime

    def call(self, state):

        #print('input shape: ',state.shape)
        #exit()
        conv1 = self.conv1(state)
        #print('Conv1 output shape:', conv1.shape)
        conv2 = self.conv2(conv1)
        #print('Conv2 output shape:', conv2.shape)
        conv3 = self.conv3(conv2)
        #print('Conv3 output shape:',conv3.shape)
        flat = self.flat(conv3)
        #print('flat : ', flat.shape)
        dense1 = self.dense1(flat)
        #print('dense1 shape: ', dense1.shape)
        actions = self.dense2(dense1)
        #print('actions: ', actions.shape)
        return actions

    def save_checkpoint(self):
      pass
      print('Saving weights...')
      self.save(self.checkpoint_file)

    def load_checkpoint(self):
      pass
      print('loading checkpoint...')
      self.load_weights(self.checkpoint_file)




In [0]:
#testing the model
#moodel = DQN(0.001, 4, 'name', (4,84,84), 'ccc')
#data = tf.random.uniform((32,4,84,84))
#moodel.call(data)

In [0]:
#env = make_env('PongNoFrameskip-v4')

In [0]:
class DQNAgent(object):
    def __init__(self, gamma, epsilon, lr, n_actions, input_dims,
                 mem_size, batch_size, eps_min=0.01, eps_dec=5e-7,
                 replace=1000, algo=None, env_name=None, chkpt_dir='tmp/dqn'):
        self.gamma = gamma
        self.epsilon = epsilon
        self.lr = lr
        self.n_actions = n_actions
        self.input_dims = input_dims
        self.batch_size = batch_size
        self.eps_min = eps_min
        self.eps_dec = eps_dec
        self.replace_target_cnt = replace
        self.algo = algo
        self.env_name = env_name
        self.chkpt_dir = chkpt_dir
        self.action_space = [i for i in range(n_actions)]
        self.learn_step_counter = 0

        self.memory = ReplayMem(mem_size, input_dims, n_actions)

        self.q_eval = DQN(self.lr, self.n_actions,
                                    input_dims=self.input_dims,
                                    name=self.env_name+'_'+self.algo+'_q_eval',
                                    chkpt_dir=self.chkpt_dir)

        self.q_next = DQN(self.lr, self.n_actions,
                                    input_dims=self.input_dims,
                                    name=self.env_name+'_'+self.algo+'_q_next',
                                    chkpt_dir=self.chkpt_dir)
        
    def choose_action(self, observation):
        if tf.random.uniform([1]) > self.epsilon:
          #take tensor and move to device
          #increase rank
          actions = self.q_eval.call(tf.expand_dims(observation, axis=0))
          #print('model output = ', actions)
          
          action = tf.argmax(actions, axis=1)
          #print('model action type:',type(action))
          #print(action)

        else:
          action = np.random.choice(self.action_space)
          #print('random action type: ', type(action))
          #print(action)
          #the enviroment wants an integer as input for an action

        return action

    def store_transition(self, state, action, reward, state_, done):
        self.memory.store_transition(state,action,reward,state_,done)

    def sample_memory(self):
        states,actions,rewards,new_states,dones = self.memory.sample_buffer(self.batch_size)

        return states, actions, rewards, new_states,dones

    def replace_target_network(self):
        if self.learn_step_counter & self.replace_target_cnt ==0:
            self.q_next = self.q_eval

    def decrement_epsilon(self):
        self.epsilon = self.epsilon - self.eps_dec \
                           if self.epsilon > self.eps_min else self.eps_min
        
    def save_models(self):
        self.q_eval.save_checkpoint()
        self.q_next.save_checkpoint()

    def load_models(self):
        self.q_eval.load_checkpoint()
        self.q_next.load_checkpoint()

    def learn(self):
        if self.memory.mem_cntr < self.batch_size:
            return
    
        self.replace_target_network()
        states,actions,rewards,states_,dones = self.sample_memory()
        optimizer = keras.optimizers.RMSprop(learning_rate=self.lr)
        indices = tf.range(self.batch_size)
        with tf.GradientTape() as tape:

            q_pred = tf.gather_nd(self.q_eval.call(states), list(zip(indices, actions)))
            q_next = tf.math.reduce_max(self.q_next.call(states_), axis=1)
        #is the code below even doing anything?
        #q_next[dones] = 0.0
            q_target = rewards + self.gamma * q_next
            loss = keras.losses.MSE(q_target, q_pred)
        gradient = tape.gradient(loss, self.q_eval.trainable_variables)
        optimizer.apply_gradients(zip(gradient, self.q_eval.trainable_variables))
        self.decrement_epsilon()

      
  
      

In [0]:
def main(num_games= 10,load_checkpoint=False, env_name='PongNoFrameskip-v4'):
    env  = make_env(env_name)
    best_score = -np.inf

    agent = DQNAgent(gamma=0.99, epsilon=1.0,lr=0.0001,input_dims=(env.observation_space.shape),
                     n_actions=env.action_space.n, mem_size=20000, eps_min=0.1, batch_size=32,replace=1000,
                     eps_dec=1e-5, chkpt_dir='models/',algo='DQNAgent', env_name=env_name)
    if load_checkpoint:
          agent.load_models()
    fname = agent.algo + '_' + agent.env_name + '_lr' + str(agent.lr) +'_' + str(num_games) + 'games'
    try:
        os.mkdir('plots')
    except:
      pass

    figure_file = 'plots/' + fname + '.png'
    n_steps = 0
    scores, eps_history,steps_array = [], [], []

    for i in range(num_games):
          done = False
          observation = env.reset()
          score = 0
          while not done:
                action = agent.choose_action(observation)
                observation_,reward,done,info = env.step(action)
                score += reward
                if not load_checkpoint:
                      agent.store_transition(observation, action, reward,observation_, int(done))
                      agent.learn()

                observation = observation_
          scores.append(score)
          steps_array.append(n_steps)
          avg_score = np.mean(scores[-100:])
          print('episode: ', i,'score: ', score,
             ' average score %.1f' % avg_score, 'best score %.2f' % best_score,
            'epsilon %.2f' % agent.epsilon, 'steps', n_steps)
          
          if avg_score > best_score:
            #if not load_checkpoint:
            #    agent.save_models()
              best_score = avg_score

          eps_history.append(agent.epsilon)
          if load_checkpoint and n_steps >= 18000:
              break

    x = [i+1 for i in range(len(scores))]
    plot_learning_curve(steps_array, scores, eps_history, figure_file)

      



In [9]:
main(300)

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>


InvalidArgumentError: ignored